# Modèle 1 : Implementation sentence transformers étude complementaire (Architecture 3)

Dans l'étude précédente nous nous avons fais ces choix :

- **Sens sémentique de la requête de l'utilisateur :** Nous avons fais pareil que pour les autres études, nous calculons un vecteur pour chaque phrases et nous faisons la moyenne. L'idée était d'avoir le sens sémentique globale de la requête.

- **Sens sémentique des solutions :** Comparer chaque section indépendament avec l'entrée de l'utilisateur et ne garder que le max de la correspondance sémentique. (Le titre, la description, etc)

## Imports

In [ ]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [4]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Fonctions inchangées de l'ancienne étude

In [5]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def pre_processing(texte):
    # Nettoyer HTML Tags
    texte = BeautifulSoup(texte, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    texte = re.sub(r'&nbsp;\.', '', texte)

    # Accents
    texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

    # Retirer les numéros
    texte = re.sub(r'\b\d+\b', '', texte)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(texte)
    phrases_propres = []
    for phrase in doc.sents:
        tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
        phrase_propre = ' '.join(tokens)
        if phrase_propre:
            phrases_propres.append(phrase_propre + ".")  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(phrases_propres)

    return cleaned_text

## Fonctions adaptées

In [6]:
def pre_processing_sections(sections):
    clean_sections = []

    for texte in sections :
        # Nettoyer HTML Tags
        texte = BeautifulSoup(texte, 'html.parser').get_text()

        # Remplacer "&nbsp;." par rien
        texte = re.sub(r'&nbsp;\.', '', texte)

        # Accents
        texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

        # Retirer les numéros
        texte = re.sub(r'\b\d+\b', '', texte)

        # Tokenization, Lemmatization, Removing Stopwords, Lowercase
        doc = nlp(texte)
        phrases_propres = []
        for phrase in doc.sents:
            tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
            phrase_propre = ' '.join(tokens)
            if phrase_propre:
                phrases_propres.append(phrase_propre + "")

        # Joining the cleaned sentences back into a single string
        cleaned_text = ' '.join(phrases_propres)

        clean_sections.append(cleaned_text)

    return clean_sections

In [7]:
def load_and_merge_data(csv_file = '../data/solutions.csv'):
    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)
    # Renommer les colonnes
    df.columns = ['id_solution', 'categorie', 'texte']
    # Filtrer les lignes pour les catégories spécifiées
    categories_specifiees = [1, 2, 5, 6]
    df_filtre = df[df['categorie'].isin(categories_specifiees)]
    # Pivoter les données pour obtenir les colonnes 'titre', 'definition', 'application' et 'bilan énergie'
    df_pivot = df_filtre.pivot(index='id_solution', columns='categorie', values='texte').reset_index()
    # Renommer les colonnes
    df_pivot.columns = ['id_solution', 'titre', 'definition', 'application', 'bilan_energie']

    # Gérer les valeurs NaN lors de la fusion des colonnes
    def combine_text(row):
        text_parts = [row[col] for col in colonnes if pd.notnull(row[col])]
        return text_parts
    
    # Sélectionner uniquement les colonnes 'id_solution' et les champs requis
    colonnes = ['titre', 'definition', 'application', 'bilan_energie']
    df_pivot['champs'] = df_pivot.apply(combine_text, axis=1)
    df_final = df_pivot[['id_solution', 'champs']]
    
    # Convertir en liste de listes pour chaque ligne
    result = df_final.values.tolist()
    
    return result

In [8]:
# NOUVELLE FONCTION
def calculate_section_embedding(sections):
    embeddings = []
    for section in sections :
        section_average_embedding = calculate_average_embedding(section)
        embeddings.append(section_average_embedding)
    return embeddings

In [9]:
def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_section_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_section_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

In [28]:
def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)


    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)

        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1).astype(np.float64), embeddings_array.astype(np.float64))

        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)

    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

## Tests

debut debug

In [8]:
# DEBUG
df_solutions = load_and_merge_data()
df_solutions[0]

[2,
 ['Installation frigorifique négative de type cascade utilisant du CO2',
  "Mise en place d'une installation frigorifique négative de type cascade utilisant du CO2 comme fluide frigorigène.",
  'Pour être éligible à CEE, la mise en place doit être effectuée par un professionnel et appliquée dans des locaux de commerce de distribution alimentaire de surface de vente inférieure à 5000 m². Comparé aux autres fluides frigorigènes, le CO2 est un fluide']]

In [9]:
# Convertir la liste de listes en DataFrame pandas
df_solutions = pd.DataFrame(df_solutions, columns=['id_solution', 'text'])

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

# Appliquer le traitement à la colonne "text" de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing_sections)

/tmp/ipykernel_38282/3329045323.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


In [10]:
df_solutions['clean_text'][0]

['installation frigorifique negativ type cascade utiliser co2',
 'mise place installation frigorifique negativ type cascade utiliser co2 fluide frigorigene',
 'eligibl cee mise place effectuee professionnel appliquee local commerce distribution alimentaire surface vente inferieure m. compare fluide frigorigene co2 fluide']

In [11]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [16]:
df_solutions = df_solutions[:5]

In [17]:
embeddings = df_solutions['clean_text'].apply(calculate_section_embedding)
    
# Créer un nouveau DataFrame avec id_solution et les embeddings
new_data = {
    'id_solution': df_solutions['id_solution'],
    'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
}

# Créer un DataFrame à partir des nouvelles données
df_embeddings = pd.DataFrame(new_data)

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

n = 2

print(df_embeddings['id_solution'][n])
for i in range (len(df_embeddings['text_embedding'][n])):
    print(df_embeddings['text_embedding'][n][i])

4
[-1.03763171e-01 -7.16285855e-02 -1.63476672e-02  6.78768605e-02
 -8.02959409e-03  5.81439361e-02 -3.03040873e-02  3.23121212e-02
  4.81155477e-02  1.96518116e-02  2.75819711e-02 -5.73833194e-03
  3.65241505e-02  1.42083149e-02  3.04668560e-04 -6.38149232e-02
 -4.43232618e-02  1.13844663e-01 -6.77734800e-03  4.43911701e-02
  8.71114992e-03  3.50807682e-02  2.83155106e-02  5.14473356e-02
  4.81382245e-03 -1.06881492e-01  5.88109754e-02  4.09756340e-02
  8.38981494e-02 -1.98878162e-02  1.02400035e-01  1.17959604e-01
  6.19233064e-02 -4.35718335e-02  3.09914574e-02 -1.46442289e-02
 -2.59205922e-02  5.52042089e-02 -6.03535101e-02  4.10556979e-02
  3.79193909e-02 -1.28711030e-01  7.90470913e-02  3.45919728e-02
 -1.27998218e-01 -8.40152428e-02 -2.93415952e-02  1.34685065e-03
  3.68741043e-02  9.29871108e-03  2.17929613e-02  1.80443183e-01
 -1.36482388e-01  6.15665093e-02  1.06387846e-01 -1.39018029e-01
 -4.53484580e-02 -5.48882410e-02  1.89979017e-01 -1.09111637e-01
 -5.13241719e-03 -2.542

fin debug

In [11]:
df_solutions = load_and_merge_data()

# Convertir la liste de listes en DataFrame pandas
df_solutions = pd.DataFrame(df_solutions, columns=['id_solution', 'text'])

# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")

# Appliquer le traitement à la colonne "text" de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing_sections)

# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

/tmp/ipykernel_75658/3329045323.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


Génère embedding

In [30]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

Les embeddings ont été storer avec succès.


Test inférence

In [31]:
# Malheureusement nous n'avons pas reçu le domaine d'activité correspondant aux 
# requetes avec Kerdos.Et pour certaines requetes nous n'avons pas l'Id_solution non
# plus, dans ce cas il est remplacé par -1.
dataset_test_kerdos = [
    ["Id_solution", "Domaine_activite", "Description"],
    [54, "", "Moteur à haut rendement"], # Titre de sol 54
    [156, "", "Remplacement des outils pneumatiques par des outils électriques lorsque c'est possible "], # Titre de sol 156
    [724, "","C'est quoi la HP flottante ?"],
    [914, "", "Je voudrais dimensionner un panneau solaire."],
    [719, "", "Quel gain pour un variateur de vitesse ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"],
    [-1, "", "J'aimerais avoir une régulation optimisée de mon groupe froid"],
    [-1, "", "Quelles sont les meilleures solutions pour l'agro-alimentaire ?"],
    [-1, "", "Comment faire pour réduire la consommation de mon compresseur d'air comprimé ?"]
]

In [12]:
# Fonction appelé par notre utilisateur
def model_PAT(secteur, description, embedding_path) :
    # On commence par concaténer notre secteur et notre description.
    text = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_text = pre_processing(text)

    # Ensuite on cherche nos similarités 
    solutions = find_solution(clean_text, embedding_path)

    # On return une liste contenant uniquement le numéros des solutions
    id_solutions = []
    for solution in solutions :
        id_solutions.append(solution[0])

    return id_solutions

In [76]:
embedding_path = "embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl"

# On va tester sur notre dataset_test
for i in range(1,len(dataset_test_kerdos)):
    print("--------------------------------------------")
    print("Solution attendue : ", dataset_test_kerdos[i][0])
    print(model_PAT(dataset_test_kerdos[i][1], dataset_test_kerdos[i][2], embedding_path))

--------------------------------------------
Solution attendue :  54
[1024, 869, 750, 904, 887, 1088, 1468, 394, 989, 1479]
--------------------------------------------
Solution attendue :  156
[1511, 986, 886, 1466, 804, 1000, 78, 398, 803, 1465]
--------------------------------------------
Solution attendue :  724
[804, 1467, 254, 251, 754, 1473, 1062, 1468, 940, 282]
--------------------------------------------
Solution attendue :  914
[807, 231, 114, 169, 1614, 1610, 1065, 1074, 996, 995]
--------------------------------------------
Solution attendue :  719
[986, 95, 1601, 868, 1469, 890, 1030, 486, 1714, 585]
--------------------------------------------
Solution attendue :  -1
[1610, 785, 1095, 128, 411, 491, 114, 1556, 262, 231]
--------------------------------------------
Solution attendue :  -1
[781, 114, 77, 996, 773, 115, 950, 134, 1485, 785]
--------------------------------------------
Solution attendue :  -1
[980, 236, 746, 1111, 393, 1556, 907, 398, 983, 1134]
------------

# TEST FINAL - Architecture 3 : 

Test_set : patrice_test_set

In [70]:
df_solutions = load_and_merge_data()
# Convertir la liste de listes en DataFrame pandas
df_solutions = pd.DataFrame(df_solutions, columns=['id_solution', 'text'])
# Charger le modèle spaCy pour le français
nlp = spacy.load("fr_core_news_sm")
# Appliquer le traitement à la colonne "text" de notre df_solutions
df_solutions['clean_text'] = df_solutions['text'].apply(pre_processing_sections)

/tmp/ipykernel_38282/3329045323.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  texte = BeautifulSoup(texte, 'html.parser').get_text()


In [26]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/patrice_test_set.csv")

def test_accuracy(embedding_path, dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT("", dataset['Description'][i], embedding_path)
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
        else :
            print("mal prédit : ", dataset["id_solution"][i])
    return accuracy

## Modèle : paraphrase-multilingual-mpnet-base-v2_embeddings.pkl

In [22]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [101]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

Les embeddings ont été storer avec succès.


In [29]:
test_accuracy("embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl")

mal prédit :  719
mal prédit :  753
mal prédit :  820
mal prédit :  1452
mal prédit :  1632


0.8800000000000004

## Modèle : sentence-transformers/LaBSE

In [87]:
# Charger le modèle
model = SentenceTransformer("sentence-transformers/LaBSE")

In [88]:
# Générons notre embedding
genere_embedding(df_solutions, "embeddings/Architecture-3_sentence-transformers-LaBSE.pkl")

Les embeddings ont été storer avec succès.


In [92]:
test_accuracy("embeddings/Architecture-3_sentence-transformers-LaBSE.pkl")

mal prédit :  148
mal prédit :  230
mal prédit :  459
mal prédit :  719
mal prédit :  820
mal prédit :  914
mal prédit :  1452
mal prédit :  1602


0.8200000000000004

## Test en incluant secteur d'activité

In [106]:
# Lire le fichier CSV
df_dataset_secteur_activite = pd.read_csv("data/patrice_test_set_secteur_activite.csv")

# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

test_accuracy("embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl", dataset=df_dataset_secteur_activite)

mal prédit :  719
mal prédit :  753
mal prédit :  808
mal prédit :  820
mal prédit :  1452
mal prédit :  1632


0.8600000000000004

# Mini étude pour définir le seul à partir duquel on renvoi une solution

Pour cela nous devons modifier *find_solution*

In [40]:
def find_solution(text_to_compare, embeddings_file, min_sol, seuil, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)
    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)
        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1).astype(np.float64), embeddings_array.astype(np.float64))
        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)
    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # PARTIE A MODIFIER ---------------------------------------------------------------------
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)

    # Récupérer les id_solution et les similarités des 10 premières lignes
    solution_info = df_sorted[['id_solution', 'similarity']].head(min_sol)

    # Mettre toutes les autres solutions dans reste_info
    reste_info = df_sorted[['id_solution', 'similarity']].iloc[min_sol:]

    # Filtrer les lignes de reste_info où la similarité est supérieure au seuil
    nouvelles_solutions = reste_info[reste_info['similarity'] > seuil]

    # Ajouter ces nouvelles solutions à solution_info
    solution_info = pd.concat([solution_info, nouvelles_solutions])

    # Réinitialiser les index pour éviter les problèmes d'indexation
    solution_info = solution_info.reset_index(drop=True)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    # ----------------------------------------------------------------------------------------
    
    return solution_list

In [46]:
#text_to_compare = "C'est quoi la HP flottante ?"
text_to_compare = "Quels sont les avantages d'une pompe à chaleur à absorption ?"

# Ensuite on applique notre pré-processing
clean_text = pre_processing(text_to_compare)

find_solution(clean_text, "embeddings/Architecture-3_paraphrase-multilingual-mpnet-base-v2_embeddings.pkl", 10, 0.75)

[(1522, tensor([0.8969], dtype=torch.float64)),
 (820, tensor([0.8689], dtype=torch.float64)),
 (349, tensor([0.8673], dtype=torch.float64)),
 (812, tensor([0.8564], dtype=torch.float64)),
 (371, tensor([0.8550], dtype=torch.float64)),
 (370, tensor([0.8327], dtype=torch.float64)),
 (348, tensor([0.8327], dtype=torch.float64)),
 (711, tensor([0.8307], dtype=torch.float64)),
 (1638, tensor([0.8193], dtype=torch.float64)),
 (923, tensor([0.8190], dtype=torch.float64)),
 (212, tensor([0.8182], dtype=torch.float64)),
 (213, tensor([0.8155], dtype=torch.float64)),
 (1035, tensor([0.8154], dtype=torch.float64)),
 (831, tensor([0.8137], dtype=torch.float64)),
 (496, tensor([0.8099], dtype=torch.float64)),
 (176, tensor([0.8066], dtype=torch.float64)),
 (1120, tensor([0.8059], dtype=torch.float64)),
 (182, tensor([0.8028], dtype=torch.float64)),
 (384, tensor([0.8025], dtype=torch.float64)),
 (291, tensor([0.8024], dtype=torch.float64)),
 (482, tensor([0.8004], dtype=torch.float64)),
 (104, te